In [1]:
!pip install -q pyngrok

     |████████████████████████████████| 745 kB 20.4 MB/s 


In [3]:
!pip install -q streamlit

In [4]:
!pip install -q streamlit-ace

     |████████████████████████████████| 3.6 MB 30.6 MB/s 


In [5]:
!pip install -q fuzzywuzzy[speedup]

     |████████████████████████████████| 50 kB 5.4 MB/s 


In [6]:
!streamlit run app.py &>/dev/null&

In [7]:
from pyngrok import ngrok

In [13]:
%%writefile app.py
import streamlit as st
import re
from fuzzywuzzy import process
import json
import requests
import pandas as pd


def main():

  namesData = requests.get('https://cspinheiro.github.io/interpol.json')

  df = pd.read_csv('Endpoints.csv')
  apiNameOptions = df['N'].drop([0,0])
  apiRoutesOptions = df['Endpoint'].drop([0, 0])
  apiRoutesOptions = apiRoutesOptions.values.tolist()
  apiNameOptions = apiNameOptions.values.tolist()

  apiRoutesOptions[12] = 'https://rodrigofer89.github.io/npl.json'
  apiRoutesOptions.pop(13)
  apiNameOptions.pop(13)


  endpointOptions = {apiNameOptions[i]: apiRoutesOptions[i] for i in range(len(apiNameOptions))}

  def handleAirportSelection(selected):
    if selected.upper() == 'ALL':
      return getAllEndpointsData()
    
    route = endpointOptions[selected.upper()]
    response = requests.get(route).json()

    totalData = []
    for i in response:
      for data in response[i]:
        totalData.append(*data.values())

    return totalData

  def getAllEndpointsData():
    
    totalData = []

    for endpoint in endpointOptions.values():
      response = requests.get(endpoint).json()
      for option in response:
        for data in response[option]:
          totalData.append(*data.values())

    return totalData

  def handleBlockedPassports(passportNumber, airport):

    passport = re.findall('[0-9]+', passportNumber)

    blockedData = handleAirportSelection(airport)

    if int(passport[0]) in blockedData:
      return 'Proibido'
    
    return 'Permitido'

  def handleBlockedPersons(name):

    blockedNamesData = namesData.json()['interpol']

    blocked = []

    for data in blockedNamesData:
      for i, v in data.items():
        blocked.append(v)

    searchList = process.extract(str(name), blocked)

    for i in searchList:
      if i[1] >= 72:
        return 'Proibido'

      return 'Permitido'


  def handleImigrate(name, passportNumber, airport):

      passport = handleBlockedPassports(passportNumber, airport)

      person = handleBlockedPersons(name)

      if passport == 'Proibido' or person == 'Proibido':
         st.error("Você não está permitido entrar nesse País")
         return
      
      st.success("Você pode entrar nesse País")
      return

  st.set_page_config(page_title="Aeroporto", page_icon="✈️", layout="centered")
  st.title("📋 Segurança do Aeroporto")
  form = st.form(key="form")

  apiNameOptions.append('ALL')

  with form:
    name = st.text_input("Nome Completo:")
    cols = st.columns((1, 1))
    passport = cols[0].text_input("Número do Passaporte:")
    airport = cols[1].selectbox("Escolha o Aeroporto:", apiNameOptions)
    submitted = st.form_submit_button(label="Verificar")

  if submitted:
    if not name or not passport:
      st.warning("Preencha todos os Campos!")
    else :
      handleImigrate(name, passport, airport)

if __name__ == '__main__':
  main()

Overwriting app.py


In [14]:
!streamlit run app.py & npx localtunnel --port 8501

2022-04-21 18:41:15.915 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.415s
your url is: https://short-yak-2.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.245.210.145:8501

  Stopping...
^C
